In [1]:
import openai, os
openai.api_key = ''
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm

In [3]:
# df = pd.read_json("/home/selvansr/capstone/DPR/outputs/2023-03-21/09-36-26/result_head2.0.json")
# df = pd.read_json("/home/selvansr/capstone/DPR/outputs/2023-03-21/08-52-35/result_tail2.0.json")

# df_1k = df.sample(1000, random_state=0).reset_index(drop=True)
# df_1k.head()

,question,answers,ctxs
0,What is the main subject of Someone's Knocking...,[serial killer],"[{'id': 'wiki:224490', 'title': 'Knock-knock j..."
1,Which conflict was David Peel Yates involved in?,[World War II],"[{'id': 'wiki:17380406', 'title': 'Rob Ford co..."
2,Who developed Akiba's Trip: Undead & Undressed?,[Acquire],"[{'id': 'wiki:17528252', 'title': 'Akiba's Tri..."
3,"Which military branch is Lewis Burwell Puller,...",[United States Marine Corps],"[{'id': 'wiki:1946530', 'title': 'Private mili..."
4,What is Robert Mardian's gender?,[male],"[{'id': 'wiki:14882303', 'title': 'Marra langu..."


In [4]:
# df_1k.to_csv("baseline_data/head2.0_1ksampled.csv", index=False)

In [2]:
df_1k = pd.read_csv("/home/selvansr/capstone/wikipedia/baseline_data/head2.0_1ksampled_alias.csv")
df_1k.head()

,Unnamed: 0,question,answers,ctxs,s1,s1_ID,property,property_ID,s2,s2_ID,s2_alias
0,1,Which conflict was David Peel Yates involved in?,['World War II'],"[{'id': 'wiki:17380406', 'title': 'Rob Ford co...",David Peel Yates,Q5235392,conflict,P607,['World War II'],['Q362'],"[['WW2', 'WWII', 'WW 2', 'WW II', 'World War T..."
1,2,Who developed Akiba's Trip: Undead & Undressed?,['Acquire'],"[{'id': 'wiki:17528252', 'title': ""Akiba's Tri...",Akiba's Trip: Undead & Undressed,Q15147454,developer,P178,['Acquire'],['Q481968'],"[['ACQUIRE Corp.', 'Acquire (game developer)']]"
2,3,"Which military branch is Lewis Burwell Puller,...",['United States Marine Corps'],"[{'id': 'wiki:1946530', 'title': 'Private mili...","Lewis Burwell Puller, Jr.",Q6536363,military branch,P241,['United States Marine Corps'],['Q11218'],"[['USMC', 'Marine Corps', 'US Marine Corps', '..."
3,4,What is Robert Mardian's gender?,['male'],"[{'id': 'wiki:14882303', 'title': 'Marra langu...",Robert Mardian,Q7347344,sex or gender,P21,['male'],['Q6581097'],"[['man', 'male person', 'male human', 'male ge..."
4,5,Where does H. Rap Brown reside?,"['United States Penitentiary, Tucson']","[{'id': 'wiki:5689261', 'title': 'Byron Brown'...",H. Rap Brown,Q5628517,residence,P551,"['United States Penitentiary, Tucson']",['Q7890885'],[['USP Tucson']]


In [14]:
import ast 
def get_document(ctx):
    return ctx[0]["text"]

df_1k["ctxs"] = df_1k["ctxs"].apply(ast.literal_eval)
df_1k["document"] = df_1k["ctxs"].apply(get_document)
df_1k.head()

,Unnamed: 0,question,answers,ctxs,s1,s1_ID,property,property_ID,s2,s2_ID,s2_alias,document
0,1,Which conflict was David Peel Yates involved in?,['World War II'],"[{'id': 'wiki:17380406', 'title': 'Rob Ford co...",David Peel Yates,Q5235392,conflict,P607,['World War II'],['Q362'],"[['WW2', 'WWII', 'WW 2', 'WW II', 'World War T...",in the case of inadvertence) is the sledgehamm...
1,2,Who developed Akiba's Trip: Undead & Undressed?,['Acquire'],"[{'id': 'wiki:17528252', 'title': 'Akiba's Tri...",Akiba's Trip: Undead & Undressed,Q15147454,developer,P178,['Acquire'],['Q481968'],"[['ACQUIRE Corp.', 'Acquire (game developer)']]",Akiba's Trip: Undead & Undressed Akiba's Trip:...
2,3,"Which military branch is Lewis Burwell Puller,...",['United States Marine Corps'],"[{'id': 'wiki:1946530', 'title': 'Private mili...","Lewis Burwell Puller, Jr.",Q6536363,military branch,P241,['United States Marine Corps'],['Q11218'],"[['USMC', 'Marine Corps', 'US Marine Corps', '...","often referred as LMB, is an Private Military ..."
3,4,What is Robert Mardian's gender?,['male'],"[{'id': 'wiki:14882303', 'title': 'Marra langu...",Robert Mardian,Q7347344,sex or gender,P21,['male'],['Q6581097'],"[['man', 'male person', 'male human', 'male ge...","can be reduced to a single verb complex, that ..."
4,5,Where does H. Rap Brown reside?,"['United States Penitentiary, Tucson']","[{'id': 'wiki:5689261', 'title': 'Byron Brown'...",H. Rap Brown,Q5628517,residence,P551,"['United States Penitentiary, Tucson']",['Q7890885'],[['USP Tucson']],"raised in Hollis, a southeastern neighborhood ..."


In [15]:
# df_new = df_1k[~df_1k["document"].isna()]
df_new = df_1k
len(df_new)

999

In [103]:
# df_new.to_csv("outputs/sampled_tail2.0.csv", index=False)

In [16]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)

In [17]:
prompt = '''Answer the question. You can use the document for additional context:

Question: Where is Nelson's Pillar located?
Document: "Nelson's Pillar (also known as the Nelson Pillar or simply the Pillar) was a large granite column capped by a statue of Horatio Nelson, built in the centre of what was then Sackville Street (later renamed O'Connell Street) in Dublin, Ireland"
Factual Answer: 'Dublin'

Question: Who directed 13 going on 30? 
Document: "13 Going on 30 (released as Suddenly 30 in some countries) is a 2004 American fantasy romantic comedy[3] film written by Cathy Yuspa and Josh Goldsmith, directed by Gary Winick, and starring Jennifer Garner about a 13-year-old girl in 1987 who dreams of being popular."
Factual Answer: 'Gary Winick'

Question: {}
Document: {}
Factual Answer: '''

In [19]:
# df_new = df_new[["question", "document", "answers"]]
df_new = df_new[["question", "document", "answers"]]
df_new

,question,document,answers
0,Which conflict was David Peel Yates involved in?,in the case of inadvertence) is the sledgehamm...,['World War II']
1,Who developed Akiba's Trip: Undead & Undressed?,Akiba's Trip: Undead & Undressed Akiba's Trip:...,['Acquire']
2,"Which military branch is Lewis Burwell Puller,...","often referred as LMB, is an Private Military ...",['United States Marine Corps']
3,What is Robert Mardian's gender?,"can be reduced to a single verb complex, that ...",['male']
4,Where does H. Rap Brown reside?,"raised in Hollis, a southeastern neighborhood ...","['United States Penitentiary, Tucson']"
...,...,...,...
994,Which health specialty is Goldenhar syndrome r...,"are incomplete development of the ear, nose, s...",['medical genetics']
995,Who is the record label of Wanda Sá?,Wanda (band) Wanda is an Austrian indie pop ba...,['Capitol Records']
996,What industry does Kyivstar belong to?,Kyivstar Kyivstar () is a Ukrainian telecommun...,"['telecommunications industry', 'mobile phone ..."
997,What is Armin Joseph Deutsch's field of work?,Daniel Kehlmann Daniel Kehlmann (born 13 Janua...,['astronomy']


In [23]:
api_response = Parallel(n_jobs=15, backend='multiprocessing')(delayed(completion_with_backoff)(model="text-davinci-003", prompt=prompt.format(q, d, a)) \
                                                                for q,d, a in tqdm(df_new.values))

100%|██████████| 999/999 [01:09<00:00, 14.47it/s]


In [24]:
df_new["api_response"] = api_response
df_new

/home/selvansr/miniconda3/envs/sling/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,question,document,answers,api_response
0,Which conflict was David Peel Yates involved in?,in the case of inadvertence) is the sledgehamm...,['World War II'],"{'id': 'cmpl-7JFJFyZZTt8bOGfADgYpcCIYM7xRZ', '..."
1,Who developed Akiba's Trip: Undead & Undressed?,Akiba's Trip: Undead & Undressed Akiba's Trip:...,['Acquire'],"{'id': 'cmpl-7JFJFBB0D0yFSNIntHKP2x9eGJzsJ', '..."
2,"Which military branch is Lewis Burwell Puller,...","often referred as LMB, is an Private Military ...",['United States Marine Corps'],"{'id': 'cmpl-7JFJF6R51Gxdi2MFYdUiAxU8ZHYha', '..."
3,What is Robert Mardian's gender?,"can be reduced to a single verb complex, that ...",['male'],"{'id': 'cmpl-7JFJFcyESR3ftardPBC6ORAnN8N3c', '..."
4,Where does H. Rap Brown reside?,"raised in Hollis, a southeastern neighborhood ...","['United States Penitentiary, Tucson']","{'id': 'cmpl-7JFJFBrMEiiJNHqXBW9ivMYAJ6jcx', '..."
...,...,...,...,...
994,Which health specialty is Goldenhar syndrome r...,"are incomplete development of the ear, nose, s...",['medical genetics'],"{'id': 'cmpl-7JFKMk0O9OQJsjgihUuuSY4YrVRYi', '..."
995,Who is the record label of Wanda Sá?,Wanda (band) Wanda is an Austrian indie pop ba...,['Capitol Records'],"{'id': 'cmpl-7JFKMj6ViRdjaBlMKakX66TmWVNdQ', '..."
996,What industry does Kyivstar belong to?,Kyivstar Kyivstar () is a Ukrainian telecommun...,"['telecommunications industry', 'mobile phone ...","{'id': 'cmpl-7JFKMbCnP0yul7BluDvv3UxicH7Us', '..."
997,What is Armin Joseph Deutsch's field of work?,Daniel Kehlmann Daniel Kehlmann (born 13 Janua...,['astronomy'],"{'id': 'cmpl-7JFKM1UEyVoT5YMtY8ph4c9EwkNqg', '..."


In [25]:
def get_text(api):
    return(api["choices"][0]["text"])
    
df_new["gpt-3"] = df_new["api_response"].apply(get_text)

/home/selvansr/miniconda3/envs/sling/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [11]:
df_new.head()

,question,answers,api_response,gpt-3
0,Which conflict was David Peel Yates involved in?,['World War II'],"{'id': 'cmpl-752ZjKJdNgp2NKOwa4kgGbszkJCRl', '...",'World War I'
1,Who developed Akiba's Trip: Undead & Undressed?,['Acquire'],"{'id': 'cmpl-752ZjdH6Ik1pvZ4RpgF3xIDTuFqOO', '...",'Acquire'
2,"Which military branch is Lewis Burwell Puller,...",['United States Marine Corps'],"{'id': 'cmpl-752Zj8g1LMog8vV0iDZbnEg6XWozX', '...",'United States Marine Corps'
3,What is Robert Mardian's gender?,['male'],"{'id': 'cmpl-752ZjoiHtQiR8fRfuHuBpFIO8Y4oc', '...",'Male'
4,Where does H. Rap Brown reside?,"['United States Penitentiary, Tucson']","{'id': 'cmpl-752ZjxpyBW8LsPsVs7u8NVTBi9YRo', '...","'Atlanta, Georgia'"


In [26]:
df_new.to_csv("outputs/sampled_head2.0_1k_passage_withgpt.csv", index=False)

In [58]:
# evaluate on exact match
import ast

indices = []
acc = 0
for idx, row in df_new.iterrows():
    ans = row["answers"]
    gpt = row['gpt-3'][3:-2].split(",")
    gpt = [ans.lower() for ans in gpt]
    count = 0
    for i in range(len(ans)):
        if ans[i].lower() in gpt:
#             print(ans[i])
            count+=1
        else:
             indices.append(idx)
    count/=len(ans)
    acc+=count

In [59]:
acc/len(df_new)

0.173

In [64]:
incorrect_df = df_1k.iloc[indices]
incorrect_df

,question,answers,ctxs
0,What is the main subject of Self-Publishing Re...,[book review],"[{'id': 'wiki:13657927', 'title': 'Self-publis..."
1,Which conflict was 115th Infantry Division inv...,[World War I],"[{'id': 'wiki:3640772', 'title': '45th Infantr..."
2,Who developed CDC display code?,[Control Data Corporation],"[{'id': 'wiki:12809904', 'title': 'CDC?', 'tex..."
5,Where does Kandahar Five reside?,[Guantanamo Bay detention camp],"[{'id': 'wiki:10503313', 'title': 'Kandahar Fi..."
6,Which administrative territorial entity is Nor...,[Arizona],"[{'id': 'wiki:6380502', 'title': 'Chilcotin Ra..."
...,...,...,...
995,Which health specialty is Rachischisis related...,[medical genetics],"[{'id': 'wiki:4634583', 'title': 'Bariatrics',..."
996,Who is the record label of Freddy's Favorites:...,[Varèse Sarabande],"[{'id': 'wiki:6343217', 'title': 'A Nightmare ..."
997,What industry does Cine Q belong to?,[entertainment],"[{'id': 'wiki:2620028', 'title': 'QVC', 'text'..."
998,What is Association for Software Testing's fie...,[computing],"[{'id': 'wiki:8336386', 'title': 'Association ..."


In [16]:
# comment out 
incorrect_df = pd.read_csv("/home/selvansr/capstone/wikipedia/baseline_data/tail2.0_1ksampled_alias.csv") 
incorrect_df.head()

,Unnamed: 0,question,answers,ctxs,s1,s1_ID,property,property_ID,s2,s2_ID,s2_alias
0,0,What is the main subject of Self-Publishing Re...,['book review'],"[{'id': 'wiki:13657927', 'title': 'Self-publis...",Self-Publishing Review,Q22908621,main subject,P921,['book review'],['Q637866'],[['review']]
1,1,Which conflict was 115th Infantry Division inv...,['World War I'],"[{'id': 'wiki:3640772', 'title': '45th Infantr...",115th Infantry Division,Q170609,conflict,P607,['World War I'],['Q361'],"[['World War One', 'WW1', 'World War 1', 'Worl..."
2,2,Who developed CDC display code?,['Control Data Corporation'],"[{'id': 'wiki:12809904', 'title': 'CDC?', 'tex...",CDC display code,Q5009913,developer,P178,['Control Data Corporation'],['Q1129191'],[['CDC']]
3,3,Which military branch is 46th (South Devonshir...,['British Army'],"[{'id': 'wiki:9683036', 'title': '46th (South ...",46th (South Devonshire) Regiment of Foot,Q4638344,military branch,P241,['British Army'],['Q222595'],[['army of the United Kingdom']]
4,4,What is Mashudu Tshifularo's gender?,['male'],"[{'id': 'wiki:18165472', 'title': 'The Adventu...",Mashudu Tshifularo,Q64746201,sex or gender,P21,['male'],['Q6581097'],"[['man', 'male person', 'male human', 'male ge..."


In [21]:
import ast
incorrect_df["ctxs"] = incorrect_df["ctxs"].apply(ast.literal_eval)

In [23]:
def get_document(ctx):
#     if ctx[0]["has_answer"] == True:
#     print(ctx[0])
    return ctx[0]["text"]
#     else:
#         return None

incorrect_df["document"] = incorrect_df["ctxs"].apply(get_document)
incorrect_df.head()

,Unnamed: 0,question,answers,ctxs,s1,s1_ID,property,property_ID,s2,s2_ID,s2_alias,document
0,0,What is the main subject of Self-Publishing Re...,['book review'],"[{'id': 'wiki:13657927', 'title': 'Self-publis...",Self-Publishing Review,Q22908621,main subject,P921,['book review'],['Q637866'],[['review']],with readers. Libraries have also become invol...
1,1,Which conflict was 115th Infantry Division inv...,['World War I'],"[{'id': 'wiki:3640772', 'title': '45th Infantr...",115th Infantry Division,Q170609,conflict,P607,['World War I'],['Q361'],"[['World War One', 'WW1', 'World War 1', 'Worl...","almost two years of fighting totalled 124,840 ..."
2,2,Who developed CDC display code?,['Control Data Corporation'],"[{'id': 'wiki:12809904', 'title': 'CDC?', 'tex...",CDC display code,Q5009913,developer,P178,['Control Data Corporation'],['Q1129191'],[['CDC']],and an answer-key at the end. To figure out th...
3,3,Which military branch is 46th (South Devonshir...,['British Army'],"[{'id': 'wiki:9683036', 'title': '46th (South ...",46th (South Devonshire) Regiment of Foot,Q4638344,military branch,P241,['British Army'],['Q222595'],[['army of the United Kingdom']],1854 and saw action at the Battle of Alma in S...
4,4,What is Mashudu Tshifularo's gender?,['male'],"[{'id': 'wiki:18165472', 'title': 'The Adventu...",Mashudu Tshifularo,Q64746201,sex or gender,P21,['male'],['Q6581097'],"[['man', 'male person', 'male human', 'male ge...","Lastly, Mahlodi is Mashaba’s high school sweet..."


In [66]:
incorrect_df = incorrect_df[~incorrect_df["document"].isna()]
incorrect_df

,question,answers,ctxs,document
11,What is Taiwan Highway Electronic Toll Collect...,[electronic toll collection],"[{'id': 'wiki:15012306', 'title': 'Electronic ...",Electronic Toll Collection (Taiwan) Taiwan Hig...
15,What is gastric mucosa a subclass of?,[mucosa],"[{'id': 'wiki:9247475', 'title': 'Gastric muco...",of surface epithelium and gastric pits of the ...
21,Who is the creator of Hampton Court Beauties?,[Godfrey Kneller],"[{'id': 'wiki:11369271', 'title': 'Hampton Cou...",Hampton Court Beauties The Hampton Court Beaut...
25,What is priestly caste a subclass of?,[caste],"[{'id': 'wiki:5371948', 'title': 'Priestly cas...",been described as a priestly caste. In the Rus...
28,What is Civil Defense geiger counter a subclas...,[Geiger counter],"[{'id': 'wiki:5369963', 'title': 'Civil defens...","Machine Works, Universal Atomics, Anton Electr..."
...,...,...,...,...
955,Who founded Nyheter Idag?,[Chang Frick],"[{'id': 'wiki:20189643', 'title': 'Nyheter Ida...",Nyheter Idag Nyheter Idag (News Today) is a Sw...
959,Who founded Fashion Law Institute?,[Susan Scafidi],"[{'id': 'wiki:17651978', 'title': 'Fashion Law...",Fashion Law Institute Established in 2010 with...
972,What is the legal form of Dewrance & Co. Ltd?,[company],"[{'id': 'wiki:2953506', 'title': 'Deloitte', '...",with Deloitte written in black color instead o...
981,Where is the headquarters of Desperado Publish...,[Norcross],"[{'id': 'wiki:12407405', 'title': 'Desperado P...",Desperado Publishing Desperado Publishing is a...


In [35]:
prompt = '''Answer the question. You can use the document for additional context:

Question: Where is Nelson's Pillar located?
Document: "Nelson's Pillar (also known as the Nelson Pillar or simply the Pillar) was a large granite column capped by a statue of Horatio Nelson, built in the centre of what was then Sackville Street (later renamed O'Connell Street) in Dublin, Ireland"
Factual Answer: 'Dublin'

Question: Who directed 13 going on 30? 
Document: "13 Going on 30 (released as Suddenly 30 in some countries) is a 2004 American fantasy romantic comedy[3] film written by Cathy Yuspa and Josh Goldsmith, directed by Gary Winick, and starring Jennifer Garner about a 13-year-old girl in 1987 who dreams of being popular."
Factual Answer: 'Gary Winick'

Question: {}
Document: {}
Factual Answer: '''

In [36]:
in_df_gpt = incorrect_df[['question', 'document', 'answers']]
in_df_gpt.head()

,question,document,answers
0,What is the main subject of Self-Publishing Re...,with readers. Libraries have also become invol...,['book review']
1,Which conflict was 115th Infantry Division inv...,"almost two years of fighting totalled 124,840 ...",['World War I']
2,Who developed CDC display code?,and an answer-key at the end. To figure out th...,['Control Data Corporation']
3,Which military branch is 46th (South Devonshir...,1854 and saw action at the Battle of Alma in S...,['British Army']
4,What is Mashudu Tshifularo's gender?,"Lastly, Mahlodi is Mashaba’s high school sweet...",['male']


In [37]:
api_response = Parallel(n_jobs=15, backend='multiprocessing')(delayed(completion_with_backoff)(model="text-davinci-003", prompt=prompt.format(q, d, a)) \
                                                                for q, d, a in tqdm(in_df_gpt.values))


100%|██████████| 999/999 [00:35<00:00, 27.96it/s]


In [38]:
in_df_gpt["api_response"] = api_response
in_df_gpt

/home/selvansr/miniconda3/envs/sling/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,question,document,answers,api_response
0,What is the main subject of Self-Publishing Re...,with readers. Libraries have also become invol...,['book review'],"{'id': 'cmpl-753LtOloiXcdX7dWfZKr7OiGMGVRP', '..."
1,Which conflict was 115th Infantry Division inv...,"almost two years of fighting totalled 124,840 ...",['World War I'],"{'id': 'cmpl-753LtWV50bfbWFyJQc1TwlwJrgsCi', '..."
2,Who developed CDC display code?,and an answer-key at the end. To figure out th...,['Control Data Corporation'],"{'id': 'cmpl-753LtpoGZsyomQ4bt2aO8evcc5sje', '..."
3,Which military branch is 46th (South Devonshir...,1854 and saw action at the Battle of Alma in S...,['British Army'],"{'id': 'cmpl-753LtNJA93KqPCVJBwEv9T9FygOwB', '..."
4,What is Mashudu Tshifularo's gender?,"Lastly, Mahlodi is Mashaba’s high school sweet...",['male'],"{'id': 'cmpl-753LtYCWVDKqXCDvjWEqiyOyMktrM', '..."
...,...,...,...,...
994,Which health specialty is Rachischisis related...,Bariatrics Bariatrics is the branch of medicin...,['medical genetics'],"{'id': 'cmpl-753MTRPIXvewY52cViKrIm7f5rq5B', '..."
995,Who is the record label of Freddy's Favorites:...,"'youth power' film."" The film was first introd...",['Varèse Sarabande'],"{'id': 'cmpl-753MTNr9oJf4IDVpXSfg0vc8TaTIy', '..."
996,What industry does Cine Q belong to?,operations. QVC/CNR broadcasts live programmin...,['entertainment'],"{'id': 'cmpl-753MTUERBnZIBKIZ2ADkKeBMnonO0', '..."
997,What is Association for Software Testing's fie...,Association for Software Testing The Associati...,['computing'],"{'id': 'cmpl-753MTmoBOV7hQg0gO105x6WKYlUvc', '..."


In [39]:
def get_text(api):
#     print(api["choices"][0]["text"].strip("\t\n,.").lower())
    text = api["choices"][0]["text"].strip().lower()
    return text
    
in_df_gpt["gpt-3"] = in_df_gpt["api_response"].apply(get_text)

/home/selvansr/miniconda3/envs/sling/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
in_df_gpt.head()

,question,document,answers,api_response,gpt-3
0,What is the main subject of Self-Publishing Re...,with readers. Libraries have also become invol...,['book review'],"{'id': 'cmpl-753LtOloiXcdX7dWfZKr7OiGMGVRP', '...",'self-publishing'
1,Which conflict was 115th Infantry Division inv...,"almost two years of fighting totalled 124,840 ...",['World War I'],"{'id': 'cmpl-753LtWV50bfbWFyJQc1TwlwJrgsCi', '...",'european theater of operations (world war ii)'
2,Who developed CDC display code?,and an answer-key at the end. To figure out th...,['Control Data Corporation'],"{'id': 'cmpl-753LtpoGZsyomQ4bt2aO8evcc5sje', '...",'william steig'
3,Which military branch is 46th (South Devonshir...,1854 and saw action at the Battle of Alma in S...,['British Army'],"{'id': 'cmpl-753LtNJA93KqPCVJBwEv9T9FygOwB', '...",'british army'
4,What is Mashudu Tshifularo's gender?,"Lastly, Mahlodi is Mashaba’s high school sweet...",['male'],"{'id': 'cmpl-753LtYCWVDKqXCDvjWEqiyOyMktrM', '...",'unspecified'


In [41]:
len(in_df_gpt)

999

In [42]:
in_df_gpt.to_csv("/home/selvansr/capstone/wikipedia/outputs/sampled_tail2.0_1k_passage_withgpt.csv", index=False)

In [161]:
# evaluate on exact match
import ast

indices = []
acc = 0
for idx, row in in_df_gpt.iterrows():
    ans = row["answers"]
    gpt = row['gpt-3'][2:-2].split(",")
    gpt = [ans.lower() for ans in gpt]
    print(gpt)
    count = 0
    for i in range(len(ans)):
        if ans[i].lower() in gpt:
#             print(ans[i])
            count+=1
        else:
            print(ans[i])
            indices.append(idx)
    count/=len(ans)
    acc+=count

[' electronic toll collection']
electronic toll collection
['simple columnar epithelium']
mucosa
['sir godfrey kneller']
Godfrey Kneller
['russian eastern orthodox church']
caste
['geiger counter']
['mark price']
["gilling beck'", " 'gilling west'", " 'north yorkshire"]
Gilling West
["magnetic fields'", " 'accelerate or stop projectiles'", " 'focus charg"]
magnetic field
['latin alphabet']
English alphabet
['peter loehr']
['utah']
["polymers'", " 'ceramic particles'", " 'alumina'", '']
ceramic
['silicon graphics']
['pacific northwest region of modern-day canada and the united states']
Pacific Northwest
['cambridge', ' cambridgeshire', ' england']
['iron maiden']
['parliament of canada']
Canada
["one praseodymium atom'", " 'three bromine atoms"]
praseodymium
['george frideric handel']
['edward emerson barnard']
["english'", " 'french'", " 'spanish"]
English
['tesco']
["jeddah'", " 'saudi arabia"]
Jeddah
['taipei', ' taiwan']
['dian uni']
India
['james shirley']
["software development to

In [162]:
print(acc/len(in_df_gpt))
print(acc)
# manual =

0.3443396226415094
73.0


In [163]:
in_df_gpt.to_csv("outputs/sampled_tail2.0_212_passage_withgpt.csv", index=False)

In [166]:
subgraph_df = df_1k.iloc[indices]
subgraph_df

,question,answers,ctxs
11,What is Taiwan Highway Electronic Toll Collect...,[electronic toll collection],"[{'id': 'wiki:15012306', 'title': 'Electronic ..."
15,What is gastric mucosa a subclass of?,[mucosa],"[{'id': 'wiki:9247475', 'title': 'Gastric muco..."
21,Who is the creator of Hampton Court Beauties?,[Godfrey Kneller],"[{'id': 'wiki:11369271', 'title': 'Hampton Cou..."
25,What is priestly caste a subclass of?,[caste],"[{'id': 'wiki:5371948', 'title': 'Priestly cas..."
69,Where was Gilling sword discovered?,[Gilling West],"[{'id': 'wiki:20916289', 'title': 'Gilling swo..."
...,...,...,...
955,Who founded Nyheter Idag?,[Chang Frick],"[{'id': 'wiki:20189643', 'title': 'Nyheter Ida..."
959,Who founded Fashion Law Institute?,[Susan Scafidi],"[{'id': 'wiki:17651978', 'title': 'Fashion Law..."
972,What is the legal form of Dewrance & Co. Ltd?,[company],"[{'id': 'wiki:2953506', 'title': 'Deloitte', '..."
981,Where is the headquarters of Desperado Publish...,[Norcross],"[{'id': 'wiki:12407405', 'title': 'Desperado P..."


In [168]:
subgraph_df.to_csv("outputs/sampled_tail2.0_generatesubgraph.csv")

In [169]:
sub_df = pd.read_csv("outputs/tail2.0_withsubgraph141.csv")
sub_df.head()

,Unnamed: 0_x,question,answers,ctxs,Unnamed: 0_y,s1,property,s2,s1_ID,property_ID,s2_ID,subgraph
0,11,What is Taiwan Highway Electronic Toll Collect...,['electronic toll collection'],"[{'id': 'wiki:15012306', 'title': 'Electronic ...",16070,Taiwan Highway Electronic Toll Collection System,subclass of,['electronic toll collection'],Q7676540,P279,['Q1191337'],"['Taiwan highest point ""Yushan Main Peak"" ', ..."
1,15,What is gastric mucosa a subclass of?,['mucosa'],"[{'id': 'wiki:9247475', 'title': 'Gastric muco...",16338,gastric mucosa,subclass of,['mucosa'],Q383249,P279,['Q66591702'],['gastric mucosa Terminologia Anatomica 98 ID ...
2,21,Who is the creator of Hampton Court Beauties?,['Godfrey Kneller'],"[{'id': 'wiki:11369271', 'title': 'Hampton Cou...",23788,Hampton Court Beauties,creator,['Godfrey Kneller'],Q5646181,P170,['Q65317'],"['painting series subclass of ""group of paint..."
3,25,What is priestly caste a subclass of?,['caste'],"[{'id': 'wiki:5371948', 'title': 'Priestly cas...",15953,priestly caste,subclass of,['caste'],Q7242921,P279,['Q484416'],['Category:Priestly castes category\'s main to...
4,69,Where was Gilling sword discovered?,['Gilling West'],"[{'id': 'wiki:20916289', 'title': 'Gilling swo...",27344,Gilling sword,location of discovery,['Gilling West'],Q60747261,P189,['Q5562355'],"['Gilling sword image Gilling', 'archaeologica..."


In [173]:
import ast

def get_document(ctx):
    ctx = ast.literal_eval(ctx)
    if ctx[0]["has_answer"] == True:
        return ctx[0]["text"]

sub_df["document_text"] = sub_df["ctxs"].apply(get_document)

#

In [175]:
sub_df = sub_df[["question","document_text", "subgraph", "answers"]]
sub_df.head()

,question,document_text,subgraph,answers
0,What is Taiwan Highway Electronic Toll Collect...,Electronic Toll Collection (Taiwan) Taiwan Hig...,"['Taiwan highest point ""Yushan Main Peak"" ', ...",['electronic toll collection']
1,What is gastric mucosa a subclass of?,of surface epithelium and gastric pits of the ...,['gastric mucosa Terminologia Anatomica 98 ID ...,['mucosa']
2,Who is the creator of Hampton Court Beauties?,Hampton Court Beauties The Hampton Court Beaut...,"['painting series subclass of ""group of paint...",['Godfrey Kneller']
3,What is priestly caste a subclass of?,been described as a priestly caste. In the Rus...,['Category:Priestly castes category\'s main to...,['caste']
4,Where was Gilling sword discovered?,Gilling sword The Gilling sword is an Anglo-Sa...,"['Gilling sword image Gilling', 'archaeologica...",['Gilling West']


In [180]:
prompt = '''answer question using the retrieved document and additional facts:

Question: Where is Nelson's Pillar located?
Document: "Nelson's Pillar (also known as the Nelson Pillar or simply the Pillar) was a large granite column capped by a statue of Horatio Nelson, built in the centre of what was then Sackville Street (later renamed O'Connell Street) in Dublin, Ireland"
Additional facts: "[Nelson's Pillar instance of victory column, Nelson's Pillar country Republic of Ireland]"
Factual Answer: ['Sackville Street', 'Dublin', Ireland']

Question: {}
Document: {}
Additional facts: {}
Factual Answer: '''

In [181]:
api_response = Parallel(n_jobs=15, backend='multiprocessing')(delayed(completion_with_backoff)(model="text-davinci-003", prompt=prompt.format(q, d, g, a)) \
                                                                for q, d, g, a in tqdm(sub_df.values))

100%|██████████| 141/141 [00:10<00:00, 13.87it/s]


In [184]:
sub_df["api_response"] = api_response

/home/selvansr/miniconda3/envs/sling/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [185]:
def get_text(api):
    text = api["choices"][0]["text"].strip().lower()
    return text
    
sub_df["gpt-3"] = sub_df["api_response"].apply(get_text)

/home/selvansr/miniconda3/envs/sling/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [186]:
sub_df.head()

,question,document_text,subgraph,answers,api_response,gpt-3
0,What is Taiwan Highway Electronic Toll Collect...,Electronic Toll Collection (Taiwan) Taiwan Hig...,"['Taiwan highest point ""Yushan Main Peak"" ', ...",['electronic toll collection'],"{'id': 'cmpl-70AMLQ6E4p3sqqhZTiuYYt8GUWQSG', '...",electronic toll collection system
1,What is gastric mucosa a subclass of?,of surface epithelium and gastric pits of the ...,['gastric mucosa Terminologia Anatomica 98 ID ...,['mucosa'],"{'id': 'cmpl-70AMLGYW9berilmPVLw9tpyeNKMks', '...",simple columnar epithelium
2,Who is the creator of Hampton Court Beauties?,Hampton Court Beauties The Hampton Court Beaut...,"['painting series subclass of ""group of paint...",['Godfrey Kneller'],"{'id': 'cmpl-70AMLbohDWuQFZjdYtQzzc2ZrScnm', '...",sir godfrey kneller
3,What is priestly caste a subclass of?,been described as a priestly caste. In the Rus...,['Category:Priestly castes category\'s main to...,['caste'],"{'id': 'cmpl-70AMLDOfmi4NXFmlobCRVwBaoKKDd', '...","""wikimedia category"""
4,Where was Gilling sword discovered?,Gilling sword The Gilling sword is an Anglo-Sa...,"['Gilling sword image Gilling', 'archaeologica...",['Gilling West'],"{'id': 'cmpl-70AMLxn7mnOu1W0RlpT7rdTUkfu1h', '...","['gilling beck', 'gilling west', 'north yorksh..."


In [199]:
import ast

indices = []
acc = 0
for idx, row in sub_df.iterrows():
    ans = ast.literal_eval(row["answers"])
    gpt = row['gpt-3']
    count = 0
    for i in range(len(ans)):
        if ans[i].lower() in gpt or ans[i].lower() == gpt:
            print(ans[i])
            print(gpt)
            count+=1
        else:
            indices.append(idx)
    count/=len(ans)
    acc+=count

electronic toll collection
electronic toll collection system
Godfrey Kneller
sir godfrey kneller
Gilling West
['gilling beck', 'gilling west', 'north yorkshire']
magnetic field
magnetic weapons use magnetic fields to accelerate or stop projectiles, or to focus charged particle
Pacific Northwest
pacific northwest region of modern-day canada and the united states.
Canada
canada
praseodymium
praseodymium(iii) bromide is a crystalline compound
English
english, french and spanish.
Jeddah
['jeddah', 'saudi arabia', '2005']
India
the indian union in 1948.
software
metrowerks produces software development tools, including an ide, compilers, link
copyright
the current state of copyright law and its implementations.
Louisiana
pointe coupee parish, louisiana, united states
novel
the paradise novels is a novel series.
Armenia
republic of armenia
United States Navy
the operator of the uss sc-742 is the united states navy.
river
the river civilization is located in (or next to) the yellow river.
Roy

In [190]:
# for idx, row in sub_df.iterrows():
#     print(row["question"])
#     print(row["answers"])
#     print(row["gpt-3"])
#     print(row["subgraph"])

What is Taiwan Highway Electronic Toll Collection System a subclass of?
['electronic toll collection']
electronic toll collection system
['Taiwan highest point  "Yushan Main Peak" ', 'Taiwan inception 19120101', 'Taiwan Curlie ID Regional/Asia/Taiwan/', 'Taiwan capital Taipei', 'Taiwan capital Canton', 'Taiwan capital Chungking', 'Taiwan capital Nanking', 'Taiwan capital Chengdu', 'Taiwan capital Nanking', 'Taiwan capital Jingzhao', 'Taiwan capital Chungking', 'Taiwan official language  "Standard Taiwanese Mandarin" ', 'Taiwan official language  "Taiwanese Hakka" ', 'Taiwan official language  "Taiwanese Hokkien" ', 'Taiwan official language  "Amis" ', 'Taiwan official language  "Paiwan" ', 'Taiwan official language  "Kinmen dialect" ', 'Taiwan official language  "O-ku-uā" ', 'Taiwan official language  "Matsu dialect" ', 'Taiwan official language  "Taiwanese Sign Language" ', 'Taiwan official language  "Saisiyat" ', 'Taiwan official language  "Puyuma" ', 'Taiwan official language  "Atay

In [200]:
sub_df.to_csv("outputs/tail2.0_with_subgraph_withgpt.csv", index=False)